# Subject Related Data

`CB DEV NOTES`
1. Some table structures and data types changed for pyrat ingestion
2. The GUI didn't work for me. Maybe an error on my end.

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
# Local Config
import os
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

Connecting cbroz@dss-db.datajoint.io:3306


DataJoint connection (connected) cbroz@dss-db.datajoint.io:3306

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

Connecting danielmk@172.26.128.53:3306


## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [2]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject

Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## Entry via GUI

In [ ]:
"""Run this cell to load the subject entry sheet."""
mouse_gui = MouseEntrySheet()
mouse_gui.app

## Entry via `insert()`

### **WARNING** 
Only run these manual entry cells if you know what you are doing!

Strings for these values may be edited to insert single rows into corresponding tables.

### Lab

In [ ]:
lab_key = 'Beck'  # Short, unique identifier for the lab. Maximum 8 characters. Example: 'Rose'.
lab_name = 'Neuronal input-output computation during cognition'  # A longer, more descriptive name for the laboratory.
institution = 'Institute for Experimental Epileptology and Cognition Research'  # The institution the laboratory belongs to.
address = 'Venusberg-Campus 1, 53127 Bonn'  # The postal address of the laboratory.

lab.insert1((lab_key, lab_name, institution, address))

### Protocol

In [ ]:
protocol_key = 'LANUF3'  # Short, unique identifier for the protocol. Maximum 16 characters.
protocol_description = 'Another dummy protocol ID for testing purposes'  # Description of the protocol.

protocol.insert1((protocol_key, protocol_description))

### User Entry

In [8]:
# insert multiple entries
data = [{'user_id': 1, 'name': 'natashak', 'lab': lab_key},
        {'user_id': 2, 'name': 'georgejk', 'lab': lab_key}]
user.insert(data)

### Line/Mutation

In [7]:
line_id = 1  # Unique identifier for the line.
line_name = 'Gcamp6-ThyC57BL/6J-Tg(Thy1-GCaMP6s)GP4.12Dkim/J'  # Description of the line.
is_active = 1  # Whether or not the line is actively breeding.

mutation_id = 2  # Unique identifier for the mutation.
mutation_description = 'Tg(Thy1-GCaMP6s)GP4.12Dkim'  # A description of the mutation.

line.insert1((line_id, line_name, is_active))
mutation.insert1((line_id, mutation_id, mutation_description))

### Subject

In [8]:
subject_id = 'WEZ-8701'
earmark = 'G155'
sex = 'M'
birth_date = '2010-08-20'
subject_description = 'Basic Mouse'
generation = 'F2'
parents = {'m': 'Rose_ROS-0001', 'f' : 'Rose_ROS-0002'}
owner_id = 1
responsible_id = 2
line_id = 1
protocol_key = 'LANUF3'

sub.insert1((subject_id, earmark, sex, birth_date, subject_description, generation, 
             parents, owner_id, responsible_id, line_id, protocol_key))

### Subject Genotype

In [10]:
subject = 'WEZ-8701'
line_id = 1
mutation_id = 2  # Unique identifier for the mutation.
genotype = 'wt/tg'  # The target phenotype of the line.

subject_genotype.insert1((subject, line_id, mutation_id, genotype))

### Project Entry

In [ ]:
project_key = 'TEC'
project_description = 'Trace Eyeblink Conditioning'

project.insert1((project_key, project_description))

### Subject Death

In [12]:
subject = 'WEZ-8701'
death_date = '2010-12-01'
case = 'natural'

subject_death.insert1((subject, death_date, case))

## Fetch

### As table

In [14]:
user * lab

user_id,"lab short lab name, pyrat labid",name,lab_name,institution,address
1,Beck,natashak,Neuronal input-output computation during cognition,Institute for Experimental Epileptology and Cognition Research,"Venusberg-Campus 1, 53127 Bonn"
2,Beck,georgejk,Neuronal input-output computation during cognition,Institute for Experimental Epileptology and Cognition Research,"Venusberg-Campus 1, 53127 Bonn"


In [15]:
sub

subject PyRat import uses this for earmark value,earmark,sex Geschlecht,birth_date date # Geb.,subject_description,generation Generation (F2 in example sheet),parent_ids dict of parent_sex: parent_eartag,owner_id,responsible_id,line,protocol
Rose_ROS-0019,G155,M,2010-08-20,Basic Mouse,F2,=BLOB=,1,2,1,LANUF3


In [45]:
line * subject_genotype

line,subject PyRat import uses this for earmark value,mutation_id,line_name,is_active,genotype
1,Rose_ROS-0019,2,Gcamp6-ThyC57BL/6J-Tg(Thy1-GCaMP6s)GP4.12Dkim/J,1,wt/tg


In [16]:
subject_genotype

subject PyRat import uses this for earmark value,line,mutation_id,genotype
Rose_ROS-0019,1,2,wt/tg


### As dictionary

One item:

In [17]:
(sub & 'subject="Rose_ROS-0019"').fetch1()

{'subject': 'Rose_ROS-0019',
 'earmark': 'G155',
 'sex': 'M',
 'birth_date': '2010-08-20',
 'subject_description': 'Basic Mouse',
 'generation': 'F2',
 'parent_ids': {'m': 'Rose_ROS-0001', 'f': 'Rose_ROS-0002'},
 'owner_id': 1,
 'responsible_id': 2,
 'line': 1,
 'protocol': 'LANUF3'}

List of dictionaries:

In [18]:
(subject_genotype & 'subject="Rose_ROS-0019"').fetch(as_dict=True)

[{'subject': 'Rose_ROS-0019',
  'line': 1,
  'mutation_id': 2,
  'genotype': 'wt/tg'}]